<a href="https://colab.research.google.com/github/zhe0/prac/blob/main/pyspark_clusteringOnNumericalAndVector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 3.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425345 sha256=ca886c45dd866d03a818ebe972f62f4dae42f03e0488599c7f0bf94953cad10a
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [2]:
import pyspark
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer, VectorAssembler, StandardScaler
from pyspark.ml.evaluation import ClusteringEvaluator
from pyspark.ml.clustering import KMeans
from pyspark.ml.linalg import Vectors
from pyspark.sql import SparkSession
import time
import pandas as pd
from sklearn.model_selection import train_test_split

In [3]:
spark = SparkSession.builder.appName("ClusteringExample").getOrCreate()
spark

In [ ]:
# data = [
#     (1.0, Vectors.dense(3.0, 4.0), "a"),
#     (2.0, Vectors.dense(6.0, 7.0), "b"),
#     (3.0, Vectors.dense(9.0, 10.0), "c"),
#     (4.0, Vectors.dense(1.0, 2.0), "a"),
#     (5.0, Vectors.dense(4.0, 5.0), "b")
# ]
# df = spark.createDataFrame(data, ["feature1", "vector", "category"])
# category_indexer = StringIndexer(inputCol="category", outputCol="category_index")

#org: name	main_category	sub_category	image	link	ratings	no_of_ratings	discount_price	actual_price
#sel: name main_category sub_category ratings no_of_ratings discount_price actual_price



In [ ]:
assembler = VectorAssembler(
    inputCols=["feature1", "vector"],
    outputCol="features")

kmeans = KMeans(k=2, featuresCol="features")

pipeline = Pipeline(stages=[category_indexer, assembler, kmeans])

model = pipeline.fit(df)

predictions = model.transform(df)

In [ ]:
predictions.show()

+--------+----------+--------+--------------+--------------+----------+
|feature1|    vector|category|category_index|      features|prediction|
+--------+----------+--------+--------------+--------------+----------+
|     1.0| [3.0,4.0]|       a|           0.0| [1.0,3.0,4.0]|         1|
|     2.0| [6.0,7.0]|       b|           1.0| [2.0,6.0,7.0]|         0|
|     3.0|[9.0,10.0]|       c|           2.0|[3.0,9.0,10.0]|         0|
|     4.0| [1.0,2.0]|       a|           0.0| [4.0,1.0,2.0]|         1|
|     5.0| [4.0,5.0]|       b|           1.0| [5.0,4.0,5.0]|         1|
+--------+----------+--------+--------------+--------------+----------+



In [ ]:
evaluator = ClusteringEvaluator()
silhouette = evaluator.evaluate(predictions)
print("Silhouette with squared euclidean distance = " + str(silhouette))
predictions.show()

Silhouette with squared euclidean distance = 0.6120640643838152
+--------+----------+--------+--------------+--------------+----------+
|feature1|    vector|category|category_index|      features|prediction|
+--------+----------+--------+--------------+--------------+----------+
|     1.0| [3.0,4.0]|       a|           0.0| [1.0,3.0,4.0]|         1|
|     2.0| [6.0,7.0]|       b|           1.0| [2.0,6.0,7.0]|         0|
|     3.0|[9.0,10.0]|       c|           2.0|[3.0,9.0,10.0]|         0|
|     4.0| [1.0,2.0]|       a|           0.0| [4.0,1.0,2.0]|         1|
|     5.0| [4.0,5.0]|       b|           1.0| [5.0,4.0,5.0]|         1|
+--------+----------+--------+--------------+--------------+----------+



### 寵物領養率的數據分析

* PetID - Unique hash ID of pet profile
* AdoptionSpeed - Categorical speed of adoption. Lower is faster. This is the value to predict. See below section for more info.
* Type - Type of animal (1 = Dog, 2 = Cat)
* Name - Name of pet (Empty if not named)
* Age - Age of pet when listed, in months
* Breed1 - Primary breed of pet (Refer to BreedLabels dictionary)
* Breed2 - Secondary breed of pet, if pet is of mixed breed (Refer to BreedLabels dictionary)
* Gender - Gender of pet (1 = Male, 2 = Female, 3 = Mixed, if profile represents group of pets)
* Color1 - Color 1 of pet (Refer to ColorLabels dictionary)
* Color2 - Color 2 of pet (Refer to ColorLabels dictionary)
* Color3 - Color 3 of pet (Refer to ColorLabels dictionary)
* MaturitySize - Size at maturity (1 = Small, 2 = Medium, 3 = Large, 4 = Extra Large, 0 = Not Specified)
* FurLength - Fur length (1 = Short, 2 = Medium, 3 = Long, 0 = Not Specified)
* Vaccinated - Pet has been vaccinated (1 = Yes, 2 = No, 3 = Not Sure)
* Dewormed - Pet has been dewormed (1 = Yes, 2 = No, 3 = Not Sure)
* Sterilized - Pet has been spayed / neutered (1 = Yes, 2 = No, 3 = Not Sure)
* Health - Health Condition (1 = Healthy, 2 = Minor Injury, 3 = Serious Injury, 0 = Not Specified)
* Quantity - Number of pets represented in profile
* Fee - Adoption fee (0 = Free)
* State - State location in Malaysia (Refer to StateLabels dictionary)
* RescuerID - Unique hash ID of rescuer
* VideoAmt - Total uploaded videos for this pet
* PhotoAmt - Total uploaded photos for this pet
* Description - Profile write-up for this pet. The primary language used is English, with some in Malay or Chinese.


In [10]:
# 貌似要做一定的資料處理...?  NAN 空格 資料行混了
# 是我自行上傳的，資料比較多
label_col = 'AdoptionSpeed'
data = pd.read_csv('./train.csv', index_col=0)
# test_data = pd.read_csv('./test.csv', index_col=0) #沒有AdoptionSpeed
train_data,test_data=train_test_split(data,test_size=0.2)
print(len(train_data),len(test_data))
train_data.head()

11994 2999


,Name,Age,Breed1,Breed2,Gender,Color1,Color2,Color3,MaturitySize,FurLength,...,Health,Quantity,Fee,State,RescuerID,VideoAmt,Description,PetID,PhotoAmt,AdoptionSpeed
Type,,,,,,,,,,,,,,,,,,,,,
1,Jin Jin,2,307,0,2,1,0,0,2,1,...,1,1,0,41401,ee2747ce26468ec44c7194e7d1d9dad9,0,Jin Jin is the only pup of 5 that could be cau...,05f84991b,10.0,2
1,Jojo,1,307,0,2,1,2,7,1,1,...,1,1,0,41326,721008d2ad70e174ecfd501b98bd476d,0,7 weeks old puppy with a very lively & hyperac...,bbf6ed419,2.0,2
2,Singha,2,265,0,1,1,7,0,2,1,...,1,1,0,41401,095c46a3f95563ef6aeda2f59c77712d,0,I found a nice new home for him already :) tha...,1b87c512e,1.0,3
2,Xiao Bai,24,266,0,2,2,4,7,2,1,...,1,1,0,41326,c5dd3fa6810843a5806906d1991af7f4,0,location: Klang contact no.,27ef6e605,1.0,4
1,RODY,12,307,0,1,1,0,0,2,1,...,1,1,0,41336,c624610e0c1baf2caa4bd8e43b769f5c,0,FROM JB:- Looking for a home to call his own. ...,4b7262ad7,2.0,3


In [23]:
train_data[label_col].unique()

array([2, 3, 4, 0, 1])

In [11]:
#remove not help columns: RescuerID, Images, PetID
print(train_data.columns)
rm=['Name','RescuerID','PetID','Images','Description']
tra=train_data[[i for i in train_data.columns if not i in rm]]
test=test_data[[i for i in test_data.columns if not i in rm]]
print(tra.columns)

Index(['Name', 'Age', 'Breed1', 'Breed2', 'Gender', 'Color1', 'Color2',
       'Color3', 'MaturitySize', 'FurLength', 'Vaccinated', 'Dewormed',
       'Sterilized', 'Health', 'Quantity', 'Fee', 'State', 'RescuerID',
       'VideoAmt', 'Description', 'PetID', 'PhotoAmt', 'AdoptionSpeed'],
      dtype='object')
Index(['Age', 'Breed1', 'Breed2', 'Gender', 'Color1', 'Color2', 'Color3',
       'MaturitySize', 'FurLength', 'Vaccinated', 'Dewormed', 'Sterilized',
       'Health', 'Quantity', 'Fee', 'State', 'VideoAmt', 'PhotoAmt',
       'AdoptionSpeed'],
      dtype='object')


In [18]:
tra_df=spark.createDataFrame(list(tra.itertuples(index=False, name=None)), tra.columns.tolist())
test_df=spark.createDataFrame(list(test.itertuples(index=False, name=None)), test.columns.tolist())

cate=['Breed1','Breed2','Gender','Color1','Color2','Color3','MaturitySize',
      'FurLength','Vaccinated','Dewormed','Sterilized','Health']
category_indexer=StringIndexer(inputCols=cate, outputCols=[i+'_' for i in cate])

scale=[i for i in tra.columns.tolist() if i not in cate]
scaler =[]
for i in scale:
    scaler.append(StandardScaler(inputCol=i, outputCol=i+'_', withStd=True, withMean=False))

In [19]:
st=time.time()
assembler = VectorAssembler(inputCols=tra_df.columns,outputCol="features")

kmeans = KMeans(k=2, featuresCol="features")

pipeline = Pipeline(stages=[ category_indexer, assembler, kmeans])
model = pipeline.fit(tra_df)

predictions = model.transform(tra_df)
print('exec time: ',(time.time()-st))
predictions.show()

exec time:  22.96891760826111
+---+------+------+------+------+------+------+------------+---------+----------+--------+----------+------+--------+---+-----+--------+--------+-------------+-------+-------+-------+-------+-------+-------+-------------+----------+-----------+---------+-----------+-------+--------------------+----------+
|Age|Breed1|Breed2|Gender|Color1|Color2|Color3|MaturitySize|FurLength|Vaccinated|Dewormed|Sterilized|Health|Quantity|Fee|State|VideoAmt|PhotoAmt|AdoptionSpeed|Breed1_|Breed2_|Gender_|Color1_|Color2_|Color3_|MaturitySize_|FurLength_|Vaccinated_|Dewormed_|Sterilized_|Health_|            features|prediction|
+---+------+------+------+------+------+------+------------+---------+----------+--------+----------+------+--------+---+-----+--------+--------+-------------+-------+-------+-------+-------+-------+-------+-------------+----------+-----------+---------+-----------+-------+--------------------+----------+
|  2|   307|     0|     2|     1|     0|     0|  

to predict test dataset

In [20]:
# training dataset performance
evaluator = ClusteringEvaluator()
silhouette = evaluator.evaluate(predictions)
print("Silhouette with squared euclidean distance = " + str(silhouette))

Silhouette with squared euclidean distance = 0.7933820625992595


In [21]:
assembler = VectorAssembler(inputCols=test_df.columns,outputCol="features")

kmeans = KMeans(k=2, featuresCol="features")

pipeline = Pipeline(stages=[ category_indexer, assembler, kmeans])
model = pipeline.fit(test_df)

predictions = model.transform(test_df)
predictions.show()

silhouette = evaluator.evaluate(predictions)
print("Silhouette with squared euclidean distance = " + str(silhouette))

+---+------+------+------+------+------+------+------------+---------+----------+--------+----------+------+--------+---+-----+--------+--------+-------------+-------+-------+-------+-------+-------+-------+-------------+----------+-----------+---------+-----------+-------+--------------------+----------+
|Age|Breed1|Breed2|Gender|Color1|Color2|Color3|MaturitySize|FurLength|Vaccinated|Dewormed|Sterilized|Health|Quantity|Fee|State|VideoAmt|PhotoAmt|AdoptionSpeed|Breed1_|Breed2_|Gender_|Color1_|Color2_|Color3_|MaturitySize_|FurLength_|Vaccinated_|Dewormed_|Sterilized_|Health_|            features|prediction|
+---+------+------+------+------+------+------+------------+---------+----------+--------+----------+------+--------+---+-----+--------+--------+-------------+-------+-------+-------+-------+-------+-------+-------------+----------+-----------+---------+-----------+-------+--------------------+----------+
| 60|   179|     0|     2|     7|     0|     0|           1|        2|         